In [1]:
import urllib.request
import zipfile

In [2]:
url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"

file_name = "horse-or-human.zip"
training_dir = "horse-or-human/training"

# Downloads files
urllib.request.urlretrieve(url, file_name)

# Unzips into specified directory
zip_ref = zipfile.ZipFile(file_name, "r")
zip_ref.extractall(training_dir)
zip_ref.close()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1/255, # The rescale performs normalization automatically
    rotation_range = 40, # Rotates a image rnadomly either 40% right or left
    width_shift_range = 0.2, # Shifting the image horizontally(left or right) by up to 20%
    height_shift_range = 0.2, # Shifting the image vertically(up or down) by up to 20%
    shear_range = 0.2, # Cropping the image by up to 20%
    zoom_range = 0.2, # Zooming the image by up to 20%,
    horizontal_flip = True, # horizontal flipping randomly
    fill_mode = 'nearest' # When sheared or shifting the other pixels are replaced by the data of it's nearest neighbor
) 

train_generator = train_datagen.flow_from_directory(
    training_dir, # specifies which directory to pull the training data from
    target_size = (300, 300), # specify the size of the images
    class_mode = "binary" # for image classification greater than 2, use "categorical"
)

Found 1027 images belonging to 2 classes.


In [4]:
from tensorflow.keras.models import Sequential # note that the import has changed
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model = Sequential([
    Conv2D(16, (3, 3), input_shape = (300, 300, 3), activation = "relu"), 
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3, 3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3, 3), activation = "relu"),
    MaxPooling2D(2,2),
    Conv2D(64, (3, 3), activation = "relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation="relu"), 
    Dense(1, activation="sigmoid")
])

/Users/avitirto/anaconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 298, 298, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 147, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 33, 33, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,606,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,704,097 (6.50 MB)

 Trainable params: 1,704,097 (6.50 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from keras.optimizers import RMSprop
model.compile(
    optimizer = RMSprop(learning_rate=0.001), # Root Mean Squared Propagation. Ensures that learning rates don't decrease
                                    # to much by placing more focus on the more recent gradients and exponentially
                                    # decaying older ones.
    loss = "binary_crossentropy", # This is the binary cross entropy loss function. In general, cross entropy is used
                                 # to compute the loss of categorical predictions
    metrics = ["accuracy"]
)

In [7]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") > 0.95:
            print("Training Accuracy has reached 95%")
            self.model.stop_training = True

callback = myCallback()
            

In [8]:
file_name = "validation-horse-or-human.zip"
test_dir = "horse-or-human/validation"
url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"

urllib.request.urlretrieve(url, file_name)

zip_ref = zipfile.ZipFile(file_name, "r")
zip_ref.extractall(test_dir)
zip_ref.close()

In [9]:
test_datagen = ImageDataGenerator(rescale = 1/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (300, 300),
    class_mode = "binary"
)

Found 256 images belonging to 2 classes.


In [10]:
history = model.fit(
    train_generator,
    epochs = 15,
    validation_data = test_generator
)

Epoch 1/15


/Users/avitirto/anaconda3/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5192 - loss: 0.7183 - val_accuracy: 0.5000 - val_loss: 0.6890
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.6472 - loss: 0.6370 - val_accuracy: 0.5078 - val_loss: 0.8618
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 37s 987ms/step - accuracy: 0.7284 - loss: 0.5727 - val_accuracy: 0.5000 - val_loss: 1.4053
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.7650 - loss: 0.4946 - val_accuracy: 0.5000 - val_loss: 2.7880
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.7723 - loss: 0.4260 - val_accuracy: 0.5000 - val_loss: 3.2102
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.8058 - loss: 0.4255 - val_accuracy: 0.5000 - val_loss: 6.0732
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.8243 - loss: 0.4491 - val_accuracy: 0.5273 - val_loss: 2.7968
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8588 - loss: 0.3118 - val_accuracy: 0.6328 - val_loss: 1.5110


In [25]:
from tensorflow.keras.preprocessing import image
import numpy as np

path = "./images/woman1.jpg"

img = image.load_img(path, target_size = (300, 300)) # resizes the input image to a 300x300 to match the CNN input
x = image.img_to_array(img) # converts the image into a 2D array
x = np.expand_dims(x, axis = 0) # adds the third dimension to match the CNN input

image_tensor = np.vstack([x]) # stacks the image vertically to match the inputs of the CNN
classes = model.predict(image_tensor)

if(classes[0] > 0.5):
    print("The Image is of a Human")
else:
    print("The Image is of a Horse")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
The Image is of a Human
